In [ ]:
# Dont run this (running should also work but pip installing in terminal is safer)
%pip install kaggle
%pip install tensorflow

In [1]:
# Import and format dataframe (Run to Start)
import tensorflow as tf
import numpy as np
import pathlib

# Set the path to the folder containing the images
data_dir = pathlib.Path('country-flags-main/png100px')

# Load images from directory
img_height = 100
img_width = 100

# Create a dataset from the directory
flags_mnist = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels=None,  # If images aren't labeled, set this to None
    image_size=(img_height, img_width),
)

# Get filenames
filenames = list(data_dir.glob('*.png'))
filenames = sorted([str(path.name) for path in filenames])  # Sort and get names as strings

Found 255 files.


In [2]:
import json

# Load the JSON file with country codes and names
with open('country-flags-main/countries.json', 'r') as f:
    country_names = json.load(f)

labels = [fname.split('.')[0].upper() for fname in filenames]  # Customize based on naming convention
labels = [country_names.get(label, 'Unknown') for label in labels]

# Convert labels to numeric format for model training
label_to_index = {name: index for index, name in enumerate(labels)}
numeric_labels = [label_to_index[label] for label in labels]

In [3]:
# Convert labels to Tensor and pair with images
label_ds = tf.data.Dataset.from_tensor_slices(numeric_labels)
image_label_ds = tf.data.Dataset.zip((flags_mnist, label_ds))

# Optionally, shuffle and batch the dataset
train_ds = image_label_ds.shuffle(buffer_size=1000).batch(32)


TypeError: '_TensorSliceDataset' object is not subscriptable

In [48]:
from tensorflow.keras import layers, models

num_classes = len(labels)

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),  # Normalize pixel values
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

c:\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [49]:
epochs = 10
history = model.fit(
    train_ds,
    epochs=epochs
)

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, None, 100, 100, 3), dtype=float32). Expected shape (None, 100, 100, 3), but input has incompatible shape (None, None, 100, 100, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, None, 100, 100, 3), dtype=float32)
  • training=True
  • mask=None